# Predicting incorrect student answers from DataShop data
<p style="margin:30px">
    <img width=50% src="https://www.featuretools.com/wp-content/uploads/2017/12/FeatureLabs-Logo-Tangerine-800.png" alt="Featuretools" />
</p>

In this tutorial, we show how to predict whether a student will succesfully answer a problem using a dataset from [CMU DataShop](https://pslcdatashop.web.cmu.edu/). While online courses are logistically efficient, the structure can make it more difficult for a teacher to understand how students are learning in their class. To try to fill in those gaps, we can apply machine learning. However, building an accurate machine learning model requires extracting information called **features**. Finding the right features is a crucial component of both finding a satisfactory answer and understanding what to do next. The process of **feature engineering** is made simple by [Featuretools](http://www.featuretools.com).

*If you're running this notebook yourself, please download the [geometry dataset](https://pslcdatashop.web.cmu.edu/DatasetInfo?datasetId=76) into the `data` folder in this repository. You will only need the `.txt` file. The infrastructure in this notebook will work with **any** DataShop dataset, but you will need to change the file name in the first cell.*

## Highlights
* Show how to import a DataShop dataset into featuretools
* Show efficacy of automatic feature generation with these datasets
* Show how to make custom primitives for stacking

# Step 1: Load data
At the beginning of any project, it is worthwhile to take a moment to think about how your dataset is structured.

In these datasets the unique events come from `transactions`: places where a student interacts with a system. Each transaction has a `time-index`, the time at which information in a row becomes known. Furthermore, the columns of those transactions have variables that can be grouped together. 

For instance, there are only 59 distinct students for the 6778 transactions we have in the geometry dataset. Those students log in to the system and have individual sessions. We can break down problems and problem steps in a similar way.

Featuretools stores data in an `EntitySet`. This is an abstraction which allows us to hold on to not only the data itself, but also to metadata like relationships and column types.

We create an entityset structure using the `datashop_to_entityset` function in [utils](utils.py). If you're interested in how `datashop_to_entityset` is structured, there's an associated notebook [entityset_function](entityset_function.ipynb) which explains choices made in more detail.

In [1]:
# Note that each branch is a one -> many relationship

# schools       students     problems
#        \        |         /
#   classes   sessions   problem steps
#          \     |       /
#           transactions  -- attempts
#

import utils

filename = 'data/ds2174_tx_All_Data_3991_2017_1128_123859.txt'
es = utils.datashop_to_entityset(filename)
es

Entityset: Dataset
  Entities:
    problem_steps (shape = [78, 50])
    sessions (shape = [59, 3])
    students (shape = [59, 2])
    transactions (shape = [6778, 26])
    problems (shape = [20, 2])
    ...And 3 more
  Relationships:
    transactions.Step Name -> problem_steps.Step Name
    problem_steps.Problem Name -> problems.Problem Name
    transactions.Session Id -> sessions.Session Id
    sessions.Anon Student Id -> students.Anon Student Id
    transactions.Class -> classes.Class
    ...and 2 more

Our `students` entity represents that: there are only 59 rows, one for each Anonymous student ID.

In [2]:
es['students'].df.head(3)

,Anon Student Id,first_sessions_time
Anon Student Id,,
Stu_c0bf45c22dc46067350d304ce330067e,Stu_c0bf45c22dc46067350d304ce330067e,1996-02-01 00:00:00
Stu_af3a2f63bda8c1338556108cb8d519a0,Stu_af3a2f63bda8c1338556108cb8d519a0,1996-02-01 00:00:02
Stu_d7f18a5fa205a889b0c5b0b56a7127d3,Stu_d7f18a5fa205a889b0c5b0b56a7127d3,1996-02-01 00:00:02


Featuretools allows us to make new entities as grouped by categorical values. Through this process of *normalization* we have created 8 connected entities from an initial table of transactions. We can look at what is left in `transactions` after normalization.

In [3]:
es['transactions'].head(3)

,Sample Name,Transaction Id,Session Id,Time,Time Zone,Duration (sec),Student Response Type,Student Response Subtype,Tutor Response Type,Tutor Response Subtype,...,Outcome,Selection,Action,Input,Feedback Text,Feedback Classification,Help Level,Total Num Hints,Class,End Time
Transaction Id,,,,,,,,,,,,,,,,,,,,,
499a0a18d7b6d96d4ee9c16d4bead6f2,All Data,499a0a18d7b6d96d4ee9c16d4bead6f2,GEO-408d5ed7:10e14be5d3a:-8000,1996-02-01 00:00:00,US/Eastern,0,ATTEMPT,NaN,RESULT,NaN,...,0,(CIRCLE-AREA_A QUESTION1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996-02-01 00:00:00
d398b66148a76c537cba816efe946b85,All Data,d398b66148a76c537cba816efe946b85,GEO-408d5ed7:10e14be5d3a:-8000,1996-02-01 00:00:01,US/Eastern,1,ATTEMPT,NaN,RESULT,NaN,...,1,(CIRCLE-AREA_A QUESTION1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996-02-01 00:00:02
1c133061306fd8e099eb0e4f2ac21430,All Data,1c133061306fd8e099eb0e4f2ac21430,GEO-408d5ed7:10e14be5d3a:-6e40,1996-02-01 00:00:02,US/Eastern,0,ATTEMPT,NaN,RESULT,NaN,...,1,(AREA QUESTION1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,1996-02-01 00:00:02


# Step 2: Building Features
Next, we calculate a feature matrix on the `transactions` entity to try to predict the outcome of a given transaction. It's at this step that our previous setup pays off: we can automatically calculate features using data from the whole `EntitySet`. 

## Cutoff times
We are going to be generating features and doing predictive modeling on time-sensitive data. That comes with a high risk of label leakage. 

In this case, we are predicting if a student will get a particular problem correct. For a fixed problem, the feature "There exists an attempt number three" would be highly predictive of the result on attempts one and two. There can only be a third attempt if there first two attempts were wrong! In that way, storing future `attempt` information in a feature to predict `Outcome` would yield higher test accuracy than the model deserves. It's not ok to have the feature "There exists an attempt number three" while predicting attempts one and two because it contains information that can not be known at that point in time.

To circumvent that, we introduce the notion of [cutoff_times](https://docs.featuretools.com/automated_feature_engineering/handling_time.html). A `cutoff_time` has an index column and a datetime column indicating the last acceptable date we can use while generating features for a historical training example. We can also add in a label, which will be passed through Deep Feature Synthesis ([DFS](https://docs.featuretools.com/automated_feature_engineering/afe.html)) untouched so we can recover it later.

Setting cutoff times immediately mitigates the risk of fraudulently using future data, controls the number of predictions we make and controls what data is used while calculating features.


In [4]:
cutoff_times = es['transactions'].df[['Transaction Id', 'End Time', 'Outcome']]
cutoff_times.head()

,Transaction Id,End Time,Outcome
Transaction Id,,,
499a0a18d7b6d96d4ee9c16d4bead6f2,499a0a18d7b6d96d4ee9c16d4bead6f2,1996-02-01 00:00:00,0
d398b66148a76c537cba816efe946b85,d398b66148a76c537cba816efe946b85,1996-02-01 00:00:02,1
1c133061306fd8e099eb0e4f2ac21430,1c133061306fd8e099eb0e4f2ac21430,1996-02-01 00:00:02,1
3ce42cc528ee77fb2db3b84c8b00ef39,3ce42cc528ee77fb2db3b84c8b00ef39,1996-02-01 00:00:02,1
82e2897fa00c8895bea39726e8dd303a,82e2897fa00c8895bea39726e8dd303a,1996-02-01 00:00:02,0


With that in hand, we can guarentee that future values for `Outcome` won't be used for any calculations because we set the time index of `Outcome` to be after the cutoff time.

From there, we can call `ft.dfs` to generate our features and feature matrix. Deep Feature Synthesis creates features using reusable functions ([Primitives](https://docs.featuretools.com/automated_feature_engineering/primitives.html)). The algorithm attempts to combine primitives together with actual data to create a feature matrix. Here, we'll use the primitives `Sum`, `Mean`, `PercentTrue` and `Hour`. 

In [5]:
import featuretools as ft
from featuretools.primitives import Sum, Mean, Hour, PercentTrue
from featuretools.selection import remove_low_information_features

import pandas as pd
pd.options.display.max_columns = 500

fm, features = ft.dfs(entityset=es,
                      target_entity='transactions',
                      agg_primitives=[Sum, Mean, PercentTrue],
                      trans_primitives=[Hour],
                      max_depth=3,
                      approximate='2m',
                      cutoff_time=cutoff_times,
                      verbose=True)

# Encode the feature matrix using One-Hot encoding
fm_enc, f_enc = ft.encode_features(fm, features)
fm_enc = fm_enc.fillna(0)
fm_enc = remove_low_information_features(fm_enc)

# Pop the label
label = fm_enc.pop('Outcome')

fm.tail()

Built 227 features
Elapsed: 01:43 | Remaining: 00:00 | Progress: 100%|██████████|| Calculated: 61/61 cutoff times


,Outcome,Class,Selection,Attempt At Step,Student Response Type,Tutor Response Subtype,Problem View,Sample Name,Feedback Classification,Level (Unit),Feedback Text,Session Id,Input,Action,Student Response Subtype,Tutor Response Type,Time Zone,Step Name,Help Level,Total Num Hints,problem_steps.KC (Khushboo),problem_steps.KC Category (KRE_circle_area),problem_steps.CF (Factor required),problem_steps.CF (Factor repeat),problem_steps.CF (Factor figure-part),problem_steps.KC Category (Geometry),problem_steps.KC (Zhulin_Textbook11_SquareRectMerge),problem_steps.KC Category (NewModel),problem_steps.KC Category (Textbook),problem_steps.KC Category (NNEWWW),problem_steps.CF (Factor cir-quad),problem_steps.KC Category (Unique-step),problem_steps.CF (Factor circle-formula),problem_steps.CF (Factor base-formula-p),problem_steps.KC (Textbook),problem_steps.KC Category (MJB-SQRECT-Merge),problem_steps.KC (Unique-step),problem_steps.KC Category (Single-KC),problem_steps.CF (Factor add-or-m),problem_steps.KC (new KC model name),problem_steps.KC (Geometry),problem_steps.KC Category (Textbook-test),problem_steps.KC Category (new KC model name),problem_steps.CF (Factor trapezoid-part),HOUR(End Time),problem_steps.CF (Factor circle-given),problem_steps.KC (KRE_circle_area),problem_steps.CF (Factor base-or-height),problem_steps.KC (Single-KC),problem_steps.CF (Factor embeddedness),problem_steps.CF (Factor circle-goal),problem_steps.KC (NNEWWW),problem_steps.KC (New),problem_steps.KC Category (New),problem_steps.KC Category (MyKC),problem_steps.KC (MyKC),problem_steps.KC (Textbook-test),problem_steps.KC (NewModel),problem_steps.KC Category (Khushboo),problem_steps.CF (Factor embedd3-tri-reg_prob_fix),problem_steps.CF (Factor basic-shape),problem_steps.CF (Factor non-standard-orientation-or-shape),sessions.Anon Student Id,HOUR(Time),problem_steps.CF (Factor parallelogram),problem_steps.KC (MJB-SQRECT-Merge),problem_steps.KC Category (Zhulin_Textbook11_SquareRectMerge),problem_steps.CF (Factor figure-type),HOUR(Problem Start Time),classes.School,problem_steps.Problem Name,problem_steps.CF (Factor parallelogram-type),problem_steps.CF (Factor backward),sessions.SUM(transactions.Tutor Response Subtype),attempts.MEAN(transactions.Input),problem_steps.MEAN(transactions.Problem View),attempts.SUM(transactions.Help Level),problem_steps.SUM(transactions.Action),problem_steps.HOUR(first_transactions_time),problem_steps.SUM(transactions.Total Num Hints),problem_steps.SUM(transactions.Feedback Text),problem_steps.MEAN(transactions.Help Level),problem_steps.MEAN(transactions.Tutor Response Subtype),problem_steps.MEAN(transactions.Action),sessions.PERCENT_TRUE(transactions.Outcome),attempts.MEAN(transactions.Feedback Classification),problem_steps.MEAN(transactions.Feedback Text),sessions.SUM(transactions.Total Num Hints),attempts.SUM(transactions.Feedback Text),attempts.MEAN(transactions.Action),problem_steps.SUM(transactions.Duration (sec)),attempts.MEAN(transactions.Problem View),sessions.MEAN(transactions.Is Last Attempt),sessions.HOUR(first_transactions_time),attempts.SUM(transactions.Total Num Hints),sessions.MEAN(transactions.Input),sessions.SUM(transactions.Help Level),problem_steps.PERCENT_TRUE(transactions.Outcome),attempts.SUM(transactions.Input),problem_steps.MEAN(transactions.Student Response Subtype),sessions.MEAN(transactions.Duration (sec)),attempts.SUM(transactions.Tutor Response Subtype),problem_steps.MEAN(transactions.Input),sessions.MEAN(transactions.Problem View),problem_steps.MEAN(transactions.Duration (sec)),attempts.SUM(transactions.Action),attempts.MEAN(transactions.Duration (sec)),attempts.SUM(transactions.Duration (sec)),attempts.SUM(transactions.Problem View),sessions.SUM(transactions.Problem View),sessions.MEAN(transactions.Tutor Response Subtype),problem_steps.SUM(transactions.Student Response Subtype),attempts.MEAN(transactions.Tutor Response Subtype),attempts.MEAN(transactions.Total Num Hints),sessions.MEAN(transactions.Student Respon

Above, you can scroll to the right to see the 227 features we created. If you look at the column names, you can see that we've done more than individually apply primitives one at a time to the raw data. Features were stacked and combined across entities in an exhaustive way. Using Deep Feature Synthesis is powerful because it greatly increases the likelihood of finding important features while decreasing the workload of the data scientist.

# Step 3: Making predictions

Here we split the data into two parts using `train_test_split` from scikit-learn. Notice that we don't want the splitter to shuffle our data, since that has the risk to leak labels in time sensitive data. 

We can do feature selection with [Recursive Feature Elimination](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html). This recursively removes features by checking feature importances (according to some model) with smaller and smaller feature sets. Here we'll set `RFE` to select 20 features.  

In [6]:
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from utils import feature_importances
from sklearn.feature_selection import RFE

# 1. Split X and y into a train and test set
X_train, X_test, y_train, y_test = train_test_split(fm_enc, label, shuffle=False)

# 2. Select features using RFE
clf = RandomForestClassifier()
estimator = clf
selector = RFE(estimator, 20, step=1)
selector = selector.fit(X_train, y_train)
X_train.iloc[:, selector.support_].tail()

,Problem View,sessions.PERCENT_TRUE(transactions.Outcome),problem_steps.SUM(transactions.Duration (sec)),sessions.MEAN(transactions.Is Last Attempt),problem_steps.PERCENT_TRUE(transactions.Outcome),sessions.MEAN(transactions.Duration (sec)),problem_steps.MEAN(transactions.Duration (sec)),attempts.MEAN(transactions.Duration (sec)),sessions.SUM(transactions.Problem View),attempts.SUM(transactions.Is Last Attempt),problem_steps.MEAN(transactions.Is Last Attempt),attempts.PERCENT_TRUE(transactions.Outcome),sessions.SUM(transactions.Duration (sec)),problem_steps.SUM(transactions.Problem View),sessions.students.MEAN(transactions.Duration (sec)),sessions.students.MEAN(transactions.Is Last Attempt),sessions.students.PERCENT_TRUE(transactions.Outcome),problem_steps.problems.PERCENT_TRUE(transactions.Outcome),sessions.students.SUM(transactions.Is Last Attempt),sessions.students.SUM(transactions.Duration (sec))
Transaction Id,,,,,,,,,,,,,,,,,,,,
79573921efea722ce617bc0e3ee17a80,1,0.786260,233.0,0.423077,0.790323,13.092308,1.909836,13.500000,180.0,3.0,0.426230,1.000000,1702.0,144.0,13.092308,0.423077,0.786260,0.758824,55.0,1702.0
4cf4aa98103ed06bbbffc3b210f598c2,1,0.898734,3901.0,0.632911,0.762105,10.550633,4.114979,11.904068,202.0,1947.0,0.422996,0.743919,1667.0,1419.0,10.550633,0.632911,0.898734,0.786614,100.0,1667.0
7ea632493c772cddae19aef3d6bca1c4,1,0.751678,3901.0,0.358108,0.762105,9.939189,4.114979,11.904068,276.0,1947.0,0.422996,0.743919,1471.0,1419.0,9.939189,0.358108,0.751678,0.786614,53.0,1471.0
fccec97aae20afa79cbb7c186b2c72da,1,0.804734,450.0,0.595238,0.800000,9.595238,13.235294,11.904068,212.0,1947.0,0.705882,0.743919,1612.0,39.0,9.595238,0.595238,0.804734,0.820000,100.0,1612.0
b4b36d998477a36a1a8afc8dae2860ea,1,0.763889,174.0,0.370370,0.764706,8.097222,10.235294,3.849727,319.0,528.0,0.764706,0.974499,1749.0,17.0,8.097222,0.370370,0.763889,0.740000,80.0,1749.0


Finally, we can train a Random Forest Classifier to make predictions. Those predictions can be checked against our `y_test` from above, and scored with a [roc_auc_score](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). Below we'll train and score our model and output the five most important features according to this model.

In [7]:
# 3. Train a Random Forest Classifier
clf.fit(selector.transform(X_train), y_train)

# 4. Make predictions and score
preds = clf.predict(selector.transform(X_test))
print("Auc score of {:.3f}".format(roc_auc_score(y_test, preds)))

feats = feature_importances(X_train.iloc[:, selector.support_], clf)

Auc score of 0.567
Feature Importances: 
1: problem_steps.PERCENT_TRUE(transactions.Outcome)
2: problem_steps.SUM(transactions.Problem View)
3: problem_steps.SUM(transactions.Duration (sec))
4: attempts.MEAN(transactions.Duration (sec))
5: problem_steps.MEAN(transactions.Duration (sec))
-----



Let's examine a feature. The feature `problem_steps.MEAN(transactions.Duration (sec))` is the average time spent on a given problem step. It's easy to see how 'amount of time people spend on this problem' might be related to problem difficulty and ultimately the `Outcome` of a given attempt.

# Next Steps
This notebook showed how to structure your data and make predictions with machine learning. Rather than spending time creating features, it's now possible to explore the relationships and implications betweem thousands of features directly. Reasonable next steps might be to:
1. Try [plotting](#Appendix:-Plotting) some of the generated features
2. Run feature selection and tune the machine learning model
3. Explore other prediction problems on this `EntitySet`




# Appendix: Plotting
Here, we'll look at a couple of important features as created above. Because Deep Feature Synthesis generates featuers automatically, it can be hard to understand what a given feature says. Here, we'll plot two important features from the model above and match results from the model to our own intuition.

In [8]:
from bokeh.io import show, output_notebook
output_notebook()
p = utils.plot(fm,
               col1='sessions.PERCENT_TRUE(transactions.Outcome)',
               col2='problem_steps.MEAN(transactions.Duration (sec))',
               label=label,
               names=['Student success rate versus problem time', 
                      'Percentage of attempts that were correct by this student', 
                      'Average time it takes a student on this problem'])
#show(p)

Loading BokehJS ...

![](data/images/exampleimage.png)

Notice that *most* students have between a 65% and 90% rate of getting a problem right overall, and most problems take fewer than 20 seconds on average. Outside of those ranges, we can start to take educated guesses about `Outcomes`. 

If a student has historically answered less the 65% of problems correctly or the current problem took other students more than 20 seconds to do, the chances are higher that the next attempt the student makes will be wrong. Similarly, if the a student's success rate has been above 90%, chances are a lot higher they'll get the next question right as well!

We can calculate those values explicitly:

In [16]:
import numpy as np
print('If < 65% correct: {:.2f}% of problems answered correctly'.format(
    100 * np.mean(fm[fm['sessions.PERCENT_TRUE(transactions.Outcome)']<=.65]['Outcome'])))
print('If > 90% correct: {:.2f}% of problems answered correctly'.format(
    100 * np.mean(fm[
        (fm['sessions.PERCENT_TRUE(transactions.Outcome)']>=.90)]['Outcome'])))

print('Most of the data has a success rate of {:.2f}%'.format(
    100 * np.mean(fm[
        (fm['sessions.PERCENT_TRUE(transactions.Outcome)']>.65) & 
        (fm['sessions.PERCENT_TRUE(transactions.Outcome)'])<.90]['Outcome'])))

If < 65% correct: 75.89% of problems answered correctly
If > 90% correct: 82.80% of problems answered correctly
Most of the data has a success rate of 80.59%


In [12]:
print('If problem takes more than 20 seconds: {:.2f}% of problems answered correctly'.format(
    100 * np.mean(fm[
        (fm['problem_steps.MEAN(transactions.Duration (sec))']>=25)]['Outcome'])))

If problem takes more than 20 seconds: 77.98% of problems answered correctly


# Feature Labs
If you have questions, comments or concerns regarding the notebook feel free to [reach out](https://stackoverflow.com/questions/tagged/featuretools).